In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/clean_data.csv")

In [4]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [5]:
df.head()

,subtitle_id,name,clean_data
0,9251120,maybe.this.time.(2014).eng.1cd,watch any video online with open subtitle free...
1,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,oh i know that it s getting late but i don t w...
2,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,i timing and subtitle by the uncontrollable lo...
3,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,ethereal music api opensubtitles org is deprec...
4,9408707,battlebots.(2015).eng.1cd,chris oh no not the minibots yelling oh you le...


In [6]:
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()
def chunk_text(text, chunk_size=500, overlap_size=50):
    chunks = []
    words = text.split()
    start_idx = 0
    while start_idx < len(words):
        end_idx = min(start_idx + chunk_size, len(words))
        chunk = ' '.join(words[start_idx:end_idx])
        chunks.append(chunk)
        start_idx += chunk_size - overlap_size
    return chunks

# Apply chunking function to the 'clean_subtitles' column
chunk_size = 500  # Number of tokens per chunk
overlap_size = 50  # Number of tokens to overlap between chunks

df['chunks'] = df['clean_data'].progress_apply(lambda x: chunk_text(x, chunk_size, overlap_size))

100%|██████████| 24749/24749 [00:11<00:00, 2125.96it/s]


In [7]:
df.shape

(24749, 4)

In [8]:
df_explod = df.explode('chunks')

In [9]:
df_explod.head()

,subtitle_id,name,clean_data,chunks
0,9251120,maybe.this.time.(2014).eng.1cd,watch any video online with open subtitle free...,watch any video online with open subtitle free...
0,9251120,maybe.this.time.(2014).eng.1cd,watch any video online with open subtitle free...,a point oh tonio seventy snack seven seven sil...
0,9251120,maybe.this.time.(2014).eng.1cd,watch any video online with open subtitle free...,like it yes you know this is my favorite spot ...
0,9251120,maybe.this.time.(2014).eng.1cd,watch any video online with open subtitle free...,i wa suddenly given you surprised me a vacatio...
0,9251120,maybe.this.time.(2014).eng.1cd,watch any video online with open subtitle free...,presentation in fairness that s so good your p...


In [10]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [11]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
from tqdm import tqdm
tqdm.pandas()

def encode_and_convert_to_list(text):
    encoded_vector = model.encode(text).tolist()
    return encoded_vector

# Apply the function to each element of 'chunks'
df_explod['doc_vector_pretrained_bert'] = df_explod['chunks'].progress_apply(encode_and_convert_to_list)

100%|██████████| 311548/311548 [54:00<00:00, 96.16it/s] 


In [13]:
df_bert_pretrained = list(df_explod.doc_vector_pretrained_bert)


In [15]:
# Initialize empty lists
documents = []
metadatas = []
ids = []
id = 1

# Iterate over DataFrame rows
for index, row in df_explod.iterrows():
    # Append document from DataFrame to documents list
    documents.append(row['chunks'])

    # Append metadata (subtitle_id) from DataFrame to metadatas list
    metadatas.append({"subtitle_id": row['subtitle_id'], "name": row['name']})

    # Append unique ID to ids list
    ids.append(str(id))

    # Increment ID
    id += 1

In [16]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68

In [17]:
import chromadb

In [18]:
client = chromadb.PersistentClient(path="vectordb")

In [19]:
client.heartbeat()

1714096196644788486

In [20]:
collection = client.create_collection(
        name="Data",
        metadata={"hnsw:space": "cosine"}
    )

In [21]:
from tqdm import tqdm

batch_size = 5000
total_batches = (len(documents) + batch_size - 1) // batch_size

for i in tqdm(range(total_batches), desc="Adding batches"):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(documents))

    batch_doc = documents[start_idx:end_idx]
    batch_metadatas = metadatas[start_idx:end_idx]
    batch_ids = ids[start_idx:end_idx]
    embeddings = df_bert_pretrained[start_idx:end_idx]

    collection.add(
        embeddings=embeddings,
        documents=batch_doc,
        metadatas=batch_metadatas,
        ids=batch_ids
    )

Adding batches: 100%|██████████| 63/63 [23:16<00:00, 22.17s/it]


In [22]:
import chromadb
client = chromadb.PersistentClient(path="vectordb")
collection = client.get_collection(name="Data")

In [23]:
def encode_input_query(query_point, model):
    """
    Encode the query point using the provided model.
    """
    return model.encode(query_point).tolist()

In [24]:
def print_movie_info(result):
    """
    Extract movie names and subtitle IDs from the query results.
    """
    movie_info = [(meta['name'], meta['subtitle_id']) for sublist in result['metadatas'] for meta in sublist if 'name' in meta and 'subtitle_id' in meta]
    return movie_info

In [25]:
query="comedy"

In [26]:
doc_vector = encode_input_query(query, model)

In [27]:
results = collection.query(
    query_embeddings=doc_vector,
    n_results=10
)

In [28]:
movie_information = results

In [29]:
movie_information

{'ids': [['142575',
   '122568',
   '142552',
   '142550',
   '94474',
   '31717',
   '33839',
   '288399',
   '185724',
   '142563']],
 'distances': [[0.5485042333602905,
   0.5559262037277222,
   0.5560411810874939,
   0.5974487066268921,
   0.597591757774353,
   0.6000708341598511,
   0.600230872631073,
   0.6086459159851074,
   0.6115579009056091,
   0.6125510334968567]],
 'metadatas': [[{'name': 'you.laugh.but.its.true.(2011).eng.1cd',
    'subtitle_id': 9324638},
   {'name': 'best.of.standup.2022.(2022).eng.1cd', 'subtitle_id': 9378479},
   {'name': 'you.laugh.but.its.true.(2011).eng.1cd', 'subtitle_id': 9324638},
   {'name': 'you.laugh.but.its.true.(2011).eng.1cd', 'subtitle_id': 9324638},
   {'name': 'prom.pact.(2023).eng.1cd', 'subtitle_id': 9486222},
   {'name': 'american.auto.s02.e01.crisis.(2023).eng.1cd',
    'subtitle_id': 9407572},
   {'name': 'sean.lock.lockipedia.live.(2010).eng.1cd',
    'subtitle_id': 9237429},
   {'name': 'limelight.(1952).eng.1cd', 'subtitle_id': 9

In [31]:
movie_information =print_movie_info(results)

In [32]:
movie_information

[('you.laugh.but.its.true.(2011).eng.1cd', 9324638),
 ('best.of.standup.2022.(2022).eng.1cd', 9378479),
 ('you.laugh.but.its.true.(2011).eng.1cd', 9324638),
 ('you.laugh.but.its.true.(2011).eng.1cd', 9324638),
 ('prom.pact.(2023).eng.1cd', 9486222),
 ('american.auto.s02.e01.crisis.(2023).eng.1cd', 9407572),
 ('sean.lock.lockipedia.live.(2010).eng.1cd', 9237429),
 ('limelight.(1952).eng.1cd', 9507698),
 ('reboot.s01.e05.what.we.do.in.the.shadows.(2022).eng.1cd', 9262338),
 ('you.laugh.but.its.true.(2011).eng.1cd', 9324638)]